In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report
import seaborn as sns
from scipy import stats
from pathlib import Path
import pickle

In [2]:
# display all columns
pd.set_option('display.max_columns', None)


### This part aims at selecting input features for trips models and model training
The original data was getten by 1.0dataclean
* add holiday column but removes
* add rush hour column
* encode time with cyclical way
* test with 46A/145/16
* average MAPE and R2 for all routes
* generate the pickle file and read 

## open the cleaned file

In [3]:
cd tmp

/home/team9/tmp


In [4]:
ls

data/


In [5]:
cd data

/home/team9/tmp/data


In [6]:
ls

Custom_location_53_345035_-6_267261_62b5c8e6c91d98000ba01ceb.csv
rt_leavetimes_DB_2018.txt
rt_trips_DB_2018.txt
rt_vehicles_DB_2018.txt
rt_vehicles_DB_2018.zip
test_46a_dask.csv
trips_modelling.csv
weather_factorised.csv
weather_trips.csv


In [ ]:
trips_modelling = pd.read_csv('trips_modelling.csv', sep=",")

In [ ]:
trips_modelling

In [9]:
trips_modelling['HOUR_DEPARTURE'].unique()

array([23,  8, 15, 10, 13, 11, 16,  9, 19, 12,  6,  5, 17, 18, 14, 20, 22,
        7, 21, 24,  4])

In [10]:
trips_modelling.dtypes

Unnamed: 0                int64
DAYOFSERVICE             object
TRIPID                    int64
PLANNEDTIME_ARR           int64
PLANNEDTIME_DEP           int64
ACTUALTIME_ARR            int64
ACTUALTIME_DEP            int64
SUPPRESSED              float64
DAYOFWEEK                 int64
HOUR_DEPARTURE            int64
JOURNEY_TIME            float64
temp                    float64
visibility              float64
wind_speed              float64
weather_main             object
weather_description      object
weather_icon             object
LINE_DIRECTION           object
planned_journey_time      int64
error                   float64
dtype: object

### add month feature

In [11]:
trips_modelling["DAYOFSERVICE"] =  trips_modelling["DAYOFSERVICE"].astype("datetime64[ns]") 

In [12]:
trips_modelling['Month'] = trips_modelling['DAYOFSERVICE'].dt.month

## change time from dummies to sin cos
A common method for encoding cyclical time data is to transform the data into two dimensions using a sine and consine transformation.

In [13]:
trips_modelling['week_sin'] = np.sin(2 * np.pi * trips_modelling['DAYOFWEEK']/6.0)
trips_modelling['week_cos'] = np.cos(2 * np.pi * trips_modelling['DAYOFWEEK']/6.0)

In [14]:
trips_modelling['hour_sin'] = np.sin(2 * np.pi * trips_modelling['HOUR_DEPARTURE']/23.0)
trips_modelling['hour_cos'] = np.cos(2 * np.pi * trips_modelling['HOUR_DEPARTURE']/23.0)

In [15]:
trips_modelling['month_sin'] = np.sin(2 * np.pi * trips_modelling['Month']/12.0)
trips_modelling['month_cos'] = np.cos(2 * np.pi * trips_modelling['Month']/12.0)

### here need some plot

### add holiday flag

In [16]:
holidays = ['2018-01-03','2018-03-17','2018-04-18','2018-05-02','2018-06-06','2018-08-01','2018-10-31','2018-12-25','2018-12-26']

In [17]:
#holidays = ['01-03','03-17','04-18','05-02','06-06','08-01','10-31','12-25','12-26']

In [18]:
trips_modelling['holiday'] = 0

In [19]:
trips_modelling

,Unnamed: 0,DAYOFSERVICE,TRIPID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,DAYOFWEEK,HOUR_DEPARTURE,JOURNEY_TIME,temp,visibility,wind_speed,weather_main,weather_description,weather_icon,LINE_DIRECTION,planned_journey_time,error,Month,week_sin,week_cos,hour_sin,hour_cos,month_sin,month_cos,holiday
0,0,2018-02-07,6253783,87245,84600,87524,84600,NaN,2,23,2924.0,6.39,9999.0,7.2,Drizzle,light intensity drizzle,09n,68_1,2645,279.0,2,0.866025,-0.5,-2.449294e-16,1.000000,0.866025,0.500000,0
1,1,2018-02-07,6254942,35512,32100,36329,32082,NaN,2,8,4247.0,-1.61,9999.0,4.1,Clouds,scattered clouds,03d,45A_2,3412,835.0,2,0.866025,-0.5,8.169699e-01,-0.576680,0.866025,0.500000,0
2,2,2018-02-07,6259460,57261,54420,58463,54443,NaN,2,15,4020.0,4.39,9999.0,5.1,Drizzle,light intensity drizzle,09d,25A_1,2841,1179.0,2,0.866025,-0.5,-8.169699e-01,-0.576680,0.866025,0.500000,0
3,3,2018-02-07,6248240,41648,37200,42019,37538,NaN,2,10,4481.0,0.39,9999.0,5.7,Clouds,broken clouds,04d,77A_2,4448,33.0,2,0.866025,-0.5,3.984011e-01,-0.917211,0.866025,0.500000,0
4,4,2018-02-07,6251760,34768,28920,35709,28929,NaN,2,8,6780.0,-1.61,9999.0,4.1,Clouds,scattered clouds,03d,39_2,5848,932.0,2,0.866025,-0.5,8.169699e-01,-0.576680,0.866025,0.500000,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788429,1810482,2018-05-14,6765652,30626,29100,30482,29119,NaN,0,8,1363.0,12.39,9999.0,3.6,Clouds,broken clouds,04d,53_2,1526,-163.0,5,0.000000,1.0,8.169699e-01,-0.576680,0.500000,-0.866025,0
1788430,1810483,2018-05-14,6765662,65950,64800,66270,64815,NaN,0,18,1455.0,16.46,9999.0,0.5,Clouds,broken clouds,04d,53_2,1150,305.0,5,0.000000,1.0,-9.790841e-01,0.203456,0.500000,-0.866025,0
1788431,1810484,2018-05-14,6765828,28647,25800,28688,25858,NaN,0,7,2830.0,11.39,9999.0,2.6,Clouds,broken clouds,04d,45A_1,2847,-17.0,5,0.000000,1.0,9.422609e-01,-0.334880,0.500000,-0.866025,0
1788432,1810485,2018-05-14,6765849,61560,57840,61365,57859,NaN,0,16,3506.0,15.46,9999.0,2.6,Clouds,broken clouds,04d,123_2,3720,-214.0,5,0.000000,1.0,-9.422609e-01,-0.334880,0.500000,-0.866025,0


In [20]:
for i in holidays:
    trips_modelling['holiday'].loc[trips_modelling['DAYOFSERVICE'] == i] = 1

/tmp/ipykernel_14972/3670487900.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_modelling['holiday'].loc[trips_modelling['DAYOFSERVICE'] == i] = 1


### add rush hour
https://www.tomtom.com/en_gb/traffic-index/dublin-traffic/  
https://www.dublinpublictransport.ie/dublin-buses#:~:text=Frequency%20of%20buses%20varies%20by,9am%20and%205pm%2D7pm).  
https://www.irishrail.ie/en-ie/faqs/when-should-i-travel  
decide to take 7-9am 4-7pm weekday for 2018, although after covin in the morning the rush hour doesn't affect too much

In [21]:
def cal_rush(df):
    df['rush_hour'] = 0
    rush_hour = [7,8,9,16,17,18,19]
    weekends = [5,6]
    for i in rush_hour:
        df['rush_hour'].loc[df['HOUR_DEPARTURE'] == i] = 1
    
    for j in weekends:
        df['rush_hour'].loc[df['DAYOFWEEK'] == j] = 0

In [22]:
trips_modelling['rush_hour'] = 0

In [23]:
rush_hour = [7,8,9,16,17,18,19]
for i in rush_hour:
    trips_modelling['rush_hour'].loc[trips_modelling['HOUR_DEPARTURE'] == i] = 1

/tmp/ipykernel_14972/2884737302.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_modelling['rush_hour'].loc[trips_modelling['HOUR_DEPARTURE'] == i] = 1


In [24]:
weekends = [5,6]
for j in weekends:
    trips_modelling['rush_hour'].loc[trips_modelling['DAYOFWEEK'] == j] = 0

/tmp/ipykernel_14972/3958362292.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  trips_modelling['rush_hour'].loc[trips_modelling['DAYOFWEEK'] == j] = 0


In [25]:
trips_modelling

,Unnamed: 0,DAYOFSERVICE,TRIPID,PLANNEDTIME_ARR,PLANNEDTIME_DEP,ACTUALTIME_ARR,ACTUALTIME_DEP,SUPPRESSED,DAYOFWEEK,HOUR_DEPARTURE,JOURNEY_TIME,temp,visibility,wind_speed,weather_main,weather_description,weather_icon,LINE_DIRECTION,planned_journey_time,error,Month,week_sin,week_cos,hour_sin,hour_cos,month_sin,month_cos,holiday,rush_hour
0,0,2018-02-07,6253783,87245,84600,87524,84600,NaN,2,23,2924.0,6.39,9999.0,7.2,Drizzle,light intensity drizzle,09n,68_1,2645,279.0,2,0.866025,-0.5,-2.449294e-16,1.000000,0.866025,0.500000,0,0
1,1,2018-02-07,6254942,35512,32100,36329,32082,NaN,2,8,4247.0,-1.61,9999.0,4.1,Clouds,scattered clouds,03d,45A_2,3412,835.0,2,0.866025,-0.5,8.169699e-01,-0.576680,0.866025,0.500000,0,1
2,2,2018-02-07,6259460,57261,54420,58463,54443,NaN,2,15,4020.0,4.39,9999.0,5.1,Drizzle,light intensity drizzle,09d,25A_1,2841,1179.0,2,0.866025,-0.5,-8.169699e-01,-0.576680,0.866025,0.500000,0,0
3,3,2018-02-07,6248240,41648,37200,42019,37538,NaN,2,10,4481.0,0.39,9999.0,5.7,Clouds,broken clouds,04d,77A_2,4448,33.0,2,0.866025,-0.5,3.984011e-01,-0.917211,0.866025,0.500000,0,0
4,4,2018-02-07,6251760,34768,28920,35709,28929,NaN,2,8,6780.0,-1.61,9999.0,4.1,Clouds,scattered clouds,03d,39_2,5848,932.0,2,0.866025,-0.5,8.169699e-01,-0.576680,0.866025,0.500000,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1788429,1810482,2018-05-14,6765652,30626,29100,30482,29119,NaN,0,8,1363.0,12.39,9999.0,3.6,Clouds,broken clouds,04d,53_2,1526,-163.0,5,0.000000,1.0,8.169699e-01,-0.576680,0.500000,-0.866025,0,1
1788430,1810483,2018-05-14,6765662,65950,64800,66270,64815,NaN,0,18,1455.0,16.46,9999.0,0.5,Clouds,broken clouds,04d,53_2,1150,305.0,5,0.000000,1.0,-9.790841e-01,0.203456,0.500000,-0.866025,0,1
1788431,1810484,2018-05-14,6765828,28647,25800,28688,25858,NaN,0,7,2830.0,11.39,9999.0,2.6,Clouds,broken clouds,04d,45A_1,2847,-17.0,5,0.000000,1.0,9.422609e-01,-0.334880,0.500000,-0.866025,0,1
1788432,1810485,2018-05-14,6765849,61560,57840,61365,57859,NaN,0,16,3506.0,15.46,9999.0,2.6,Clouds,broken clouds,04d,123_2,3720,-214.0,5,0.000000,1.0,-9.422609e-01,-0.334880,0.500000,-0.866025,0,1


### holiday rush hour?


In [26]:
# holidays = ['2018-01-03','2018-03-17','2018-04-18','2018-05-02','2018-06-06','2018-08-01','2018-10-31','2018-12-25','2018-12-26']
# for i in holidays:
#     trips_modelling['holiday'].loc[trips_modelling['rush_hour'] == i] = 1

### deal with weather 

In [27]:
trips_modelling['weather_main'] = trips_modelling['weather_main'].apply(lambda x: 1 if (x == "Rain" or x == "Mist" or x == "Thunderstorm") else 0)

In [28]:
# calculate correlation coefficient
r, p = stats.pearsonr(trips_modelling['weather_main'], trips_modelling['JOURNEY_TIME'])
print('Pearson\'s correlation r is %s with a p-value = %s' %(r,p))

Pearson's correlation r is 0.020311769516159373 with a p-value = 1.6302925131961625e-162


In [29]:
trips_modelling['weather_main'].value_counts(dropna=False)

0    1406222
1     382212
Name: weather_main, dtype: int64

## Select input features for the models

In [30]:
# trips_modelling = trips_modelling.drop({'Unnamed: 0','DAYOFSERVICE', "TRIPID", "PLANNEDTIME_ARR", "PLANNEDTIME_DEP", "ACTUALTIME_ARR","ACTUALTIME_DEP", "SUPPRESSED", "DAYOFWEEK", "HOUR_DEPARTURE", "visibility", "wind_speed", "weather_description", "weather_icon", "date", "error", "Month",'planned_journey_time','holiday'}, axis=1)

In [31]:
trips_modelling = trips_modelling.drop({'Unnamed: 0','DAYOFSERVICE', "TRIPID", "PLANNEDTIME_ARR", "PLANNEDTIME_DEP", "ACTUALTIME_ARR","ACTUALTIME_DEP", "SUPPRESSED", "DAYOFWEEK", "HOUR_DEPARTURE", "visibility", "wind_speed", "weather_description", "weather_icon", "error", "Month",'planned_journey_time','holiday'}, axis=1)

In [32]:
trips_modelling["LINE_DIRECTION"] =  trips_modelling["LINE_DIRECTION"].astype("category") 

In [33]:
trips_modelling["weather_main"] =  trips_modelling["weather_main"].astype("category") 

In [34]:
trips_modelling.dtypes

JOURNEY_TIME       float64
temp               float64
weather_main      category
LINE_DIRECTION    category
week_sin           float64
week_cos           float64
hour_sin           float64
hour_cos           float64
month_sin          float64
month_cos          float64
rush_hour            int64
dtype: object

In [35]:
trips_modelling

,JOURNEY_TIME,temp,weather_main,LINE_DIRECTION,week_sin,week_cos,hour_sin,hour_cos,month_sin,month_cos,rush_hour
0,2924.0,6.39,0,68_1,0.866025,-0.5,-2.449294e-16,1.000000,0.866025,0.500000,0
1,4247.0,-1.61,0,45A_2,0.866025,-0.5,8.169699e-01,-0.576680,0.866025,0.500000,1
2,4020.0,4.39,0,25A_1,0.866025,-0.5,-8.169699e-01,-0.576680,0.866025,0.500000,0
3,4481.0,0.39,0,77A_2,0.866025,-0.5,3.984011e-01,-0.917211,0.866025,0.500000,0
4,6780.0,-1.61,0,39_2,0.866025,-0.5,8.169699e-01,-0.576680,0.866025,0.500000,1
...,...,...,...,...,...,...,...,...,...,...,...
1788429,1363.0,12.39,0,53_2,0.000000,1.0,8.169699e-01,-0.576680,0.500000,-0.866025,1
1788430,1455.0,16.46,0,53_2,0.000000,1.0,-9.790841e-01,0.203456,0.500000,-0.866025,1
1788431,2830.0,11.39,0,45A_1,0.000000,1.0,9.422609e-01,-0.334880,0.500000,-0.866025,1
1788432,3506.0,15.46,0,123_2,0.000000,1.0,-9.422609e-01,-0.334880,0.500000,-0.866025,1


## route selction

In [36]:
fortySixA1 = trips_modelling.loc[trips_modelling['LINE_DIRECTION']=="46A_1"]
fortySixA1 = fortySixA1.drop(columns=["LINE_DIRECTION"])

In [37]:
sixteen = trips_modelling.loc[trips_modelling['LINE_DIRECTION']=="16_1"]
sixteen = sixteen.drop(columns=["LINE_DIRECTION"])

In [38]:
onefourfive = trips_modelling.loc[trips_modelling['LINE_DIRECTION']=="145_1"]
onefourfive = onefourfive.drop(columns=["LINE_DIRECTION"])

In [39]:
fortySixA1

,JOURNEY_TIME,temp,weather_main,week_sin,week_cos,hour_sin,hour_cos,month_sin,month_cos,rush_hour
13,5149.0,3.39,1,8.660254e-01,-0.5,-9.422609e-01,-0.334880,0.866025,5.000000e-01,1
18,4960.0,-1.61,0,8.660254e-01,-0.5,9.976688e-01,-0.068242,0.866025,5.000000e-01,0
178,3537.0,7.39,1,-2.449294e-16,1.0,-2.449294e-16,1.000000,1.000000,6.123234e-17,0
179,3261.0,7.39,1,-2.449294e-16,1.0,-5.195840e-01,0.854419,1.000000,6.123234e-17,0
180,2779.0,7.39,1,-2.449294e-16,1.0,-5.195840e-01,0.854419,1.000000,6.123234e-17,0
...,...,...,...,...,...,...,...,...,...,...
1788137,4553.0,15.39,0,8.660254e-01,0.5,6.310879e-01,-0.775711,0.500000,-8.660254e-01,1
1788167,4716.0,15.39,0,8.660254e-01,0.5,-8.169699e-01,-0.576680,0.500000,-8.660254e-01,0
1788283,4122.0,17.19,0,8.660254e-01,0.5,-1.361666e-01,-0.990686,0.500000,-8.660254e-01,0
1788295,5798.0,14.39,1,8.660254e-01,0.5,-9.422609e-01,-0.334880,0.500000,-8.660254e-01,1


### Train/Test Splitting

In [40]:
# setting random state to 0 as does not matter that order is the same each time once its shuffled
# setting test_size to 30%
# using sklearns train_test_split method
train_46, test_46 = train_test_split(fortySixA1, test_size=0.3, random_state=1)
train_145, test_145 = train_test_split(onefourfive, test_size=0.3, random_state=1)
train_16, test_16 = train_test_split(sixteen, test_size=0.3, random_state=1)

In [41]:
# resetting the indexes of both to start at 0 (preserves order generated above)
train_46 = train_46.reset_index(drop=True)
test_46 = test_46.reset_index(drop=True)
train_145 = train_145.reset_index(drop=True)
test_145 = test_145.reset_index(drop=True)

train_16 = train_16.reset_index(drop=True)
test_16 = test_16.reset_index(drop=True)

In [42]:
print("Training dataset for 46A has", train_46.shape[0], "rows and test dataset has", test_46.shape[0], "rows.")
print("Training dataset for 145 has", train_145.shape[0], "rows and test dataset has", test_145.shape[0], "rows.")
print("Training dataset for 16 has", train_16.shape[0], "rows and test dataset has", test_16.shape[0], "rows.")

Training dataset for 46A has 23760 rows and test dataset has 10183 rows.
Training dataset for 145 has 18515 rows and test dataset has 7936 rows.
Training dataset for 16 has 14058 rows and test dataset has 6025 rows.


In [43]:
# verify datatypes have remained intact
test_46.dtypes

JOURNEY_TIME     float64
temp             float64
weather_main    category
week_sin         float64
week_cos         float64
hour_sin         float64
hour_cos         float64
month_sin        float64
month_cos        float64
rush_hour          int64
dtype: object

In [44]:
# predictions = [feature for feature in list(train_46.columns) if feature != 'JOURNEY_TIME' and 'MONTH' not in feature]
predictions = [feature for feature in list(train_46.columns) if feature != 'JOURNEY_TIME']
target = 'JOURNEY_TIME'

In [ ]:
predictions

In [ ]:
# used to compute all metrics
def RegressionMetrics(actualVal, predictions, num_pred, num_samples):
    # classification evaluation measures
    print("MAE:", metrics.mean_absolute_error(actualVal, predictions))
    print("MAPE:", metrics.mean_absolute_percentage_error(actualVal, predictions))
    
    print("RMSE:", metrics.mean_squared_error(actualVal, predictions)**0.5)
    r2 = metrics.r2_score(actualVal, predictions)
    print("R2:", r2)
    print("Adjusted R2:", 1 - ((1-r2) * ((num_samples-1)/(num_samples-num_pred-1))))

In [ ]:
def cal_mape(actualVal, predictions, num_pred, num_samples):
    mape = metrics.mean_absolute_percentage_error(actualVal, predictions)
    return mape

In [ ]:
def cal_r2(actualVal, predictions, num_pred, num_samples):
    r2 = metrics.r2_score(actualVal, predictions)
    return r2

### Linear Regression Modelling

#### 46A

In [ ]:
# fit the linear model
linreg = LinearRegression().fit(train_46[predictions], train_46[target])
linreg_display_df = pd.DataFrame()
linreg_display_df['Feature'] = predictions
linreg_display_df['Coefficient'] = linreg.coef_
intercept = linreg.intercept_
print("Intercept:", intercept)
print("\n Features for 46A & their coefficients")
linreg_display_df.sort_values('Coefficient', ascending=False)

In [ ]:
# use the model to predict values for training set
predictedLinRegTrain46a = linreg.predict(train_46[predictions])

In [ ]:
samples = train_46.shape[0]
predictors = train_46[predictions].shape[1]
print("\u0332".join("Regression Evaluation Measures for 46A Training Set"))
RegressionMetrics(train_46[target], predictedLinRegTrain46a, predictors, samples)

In [ ]:
predictedTest = linreg.predict(test_46[predictions])
print("\u0332".join("Regression Evaluation Measures for 46A Test Set"))
RegressionMetrics(test_46[target], predictedTest, predictors, test_46.shape[0])

#### 145

In [ ]:
# fit the linear model
linreg = LinearRegression().fit(train_145[predictions], train_145[target])
linreg_display_df = pd.DataFrame()
linreg_display_df['Feature'] = predictions
linreg_display_df['Coefficient'] = linreg.coef_
intercept = linreg.intercept_
print("Intercept:", intercept)
print("\n Features for 145 & their coefficients")
linreg_display_df.sort_values('Coefficient', ascending=False)

In [ ]:
# use the model to predict values for training set
predictedLinRegTrain145 = linreg.predict(train_145[predictions])

In [ ]:
samples = train_145.shape[0]
predictors = train_145[predictions].shape[1]
print("\u0332".join("Regression Evaluation Measures for 145 Training Set"))
RegressionMetrics(train_145[target], predictedLinRegTrain145, predictors, samples)

In [ ]:
predictedTest145 = linreg.predict(test_145[predictions])
print("\u0332".join("Regression Evaluation Measures for Test Set"))
RegressionMetrics(test_145[target], predictedTest145, predictors, test_145.shape[0])

#### 16

In [ ]:
# fit the linear model
linreg = LinearRegression().fit(train_16[predictions], train_16[target])
linreg_display_df = pd.DataFrame()
linreg_display_df['Feature'] = predictions
linreg_display_df['Coefficient'] = linreg.coef_
intercept = linreg.intercept_
print("Intercept:", intercept)
print("\n Features for 16 & their coefficients")
linreg_display_df.sort_values('Coefficient', ascending=False)

In [ ]:
# use the model to predict values for training set
predictedLinRegTrain16 = linreg.predict(train_16[predictions])

In [ ]:
samples = train_16.shape[0]
predictors = train_16[predictions].shape[1]
print("\u0332".join("Regression Evaluation Measures for 16 Training Set"))
RegressionMetrics(train_16[target], predictedLinRegTrain16, predictors, samples)

In [ ]:
predictedTest16 = linreg.predict(test_16[predictions])
print("\u0332".join("Regression Evaluation Measures for Test Set"))
RegressionMetrics(test_16[target], predictedTest16, predictors, test_16.shape[0])

### Random Forest Modelling

In [ ]:
# initialise the RandomForestRegressor
# set random state value so that results can be replicated
forestReg_46 = RandomForestRegressor(n_estimators=100, max_depth=100,oob_score=True, random_state=1)

forestReg_145 = RandomForestRegressor(n_estimators=100, max_depth=100,oob_score=True, random_state=1)

forestReg_16 = RandomForestRegressor(n_estimators=100, max_depth=100,oob_score=True, random_state=1)


# fit the model on the training set using the predictions features
# y value is now the quantiles
forestReg_46.fit(train_46[predictions], train_46[target])

forestReg_145.fit(train_145[predictions], train_145[target])

forestReg_16.fit(train_16[predictions], train_16[target])

In [ ]:
feature_importance_46 = pd.DataFrame({'feature': train_46[predictions].columns, 'importance':forestReg_46.feature_importances_})
print("Feature Importance Random Forest Regressor for 46A: ")
feature_importance_46.sort_values('importance', ascending=False)

In [ ]:
feature_importance_145 = pd.DataFrame({'feature': train_145[predictions].columns, 'importance':forestReg_145.feature_importances_})
print("Feature Importance Random Forest Regressor for 145:")
feature_importance_145.sort_values('importance', ascending=False)

In [ ]:
feature_importance_16 = pd.DataFrame({'feature': train_46[predictions].columns, 'importance':forestReg_16.feature_importances_})
print("Feature Importance Random Forest Regressor for 16:")
feature_importance_16.sort_values('importance', ascending=False)

#### 46A

In [ ]:
forestRegPredictedTrain46_10 = forestReg_46.predict(test_46[predictions].head(10))
print("First 10 predictions with Random Forest Regression for Route 46a:")
predicted_df_train_reg_46 = pd.concat([test_46[target].head(10), pd.DataFrame(forestRegPredictedTrain46_10, columns=['Predicted Time'])], axis=1)
predicted_df_train_reg_46

In [ ]:
# predict entire training dataset using random forest regressor model
forestRegPredictedTrain46 = forestReg_46.predict(train_46[predictions])

In [ ]:
print("\u0332".join("Regression Evaluation Measures for 46A Training Set"))
RegressionMetrics(train_46[target], forestRegPredictedTrain46, predictors, train_46.shape[0])

In [ ]:
# predict entire test dataset using random forest regressor model
forestRegPredictedTest46 = forestReg_46.predict(test_46[predictions])
print("\u0332".join("Regression Evaluation Measures for 46A Test Set"))
RegressionMetrics(test_46[target], forestRegPredictedTest46, predictors, test_46.shape[0])

#### 145

In [ ]:
# predict entire training dataset using random forest regressor model
forestRegPredictedTrain145 = forestReg_145.predict(train_145[predictions])

In [ ]:
print("\u0332".join("Regression Evaluation Measures for 145 Training Set"))
RegressionMetrics(train_145[target], forestRegPredictedTrain145, predictors, train_145.shape[0])

In [ ]:
# predict entire test dataset using random forest regressor model
forestRegPredictedTest145 = forestReg_145.predict(test_145[predictions])
print("\u0332".join("Regression Evaluation Measures for 145 Test Set"))
RegressionMetrics(test_145[target], forestRegPredictedTest145, predictors, test_145.shape[0])

#### 16

In [ ]:
# predict entire training dataset using random forest regressor model
forestRegPredictedTrain16 = forestReg_16.predict(train_16[predictions])

In [ ]:
print("\u0332".join("Regression Evaluation Measures for 16 Training Set"))
RegressionMetrics(train_16[target], forestRegPredictedTrain16, predictors, train_16.shape[0])

In [ ]:
# predict entire test dataset using random forest regressor model
forestRegPredictedTest16 = forestReg_16.predict(test_16[predictions])
print("\u0332".join("Regression Evaluation Measures for 16 Test Set"))
RegressionMetrics(test_16[target], forestRegPredictedTest16, predictors, test_16.shape[0])

### KNN Modelling

#### 46

In [ ]:
knn46 = KNeighborsRegressor().fit(train_46[predictions], train_46[target])

In [ ]:
# use the model to predict values for training set
predictedKNNTrain46 = knn46.predict(train_46[predictions])

In [ ]:
samples = train_46.shape[0]
predictors = train_46[predictions].shape[1]
print("\u0332".join("Regression Evaluation Measures for 46A Training Set"))
RegressionMetrics(train_46[target], predictedKNNTrain46, predictors, samples)

In [ ]:
predictedTest = knn46.predict(test_46[predictions])
print("\u0332".join("Regression Evaluation Measures for 46A Test Set"))
RegressionMetrics(test_46[target], predictedTest, predictors, test_46.shape[0])

#### 46

In [ ]:
knn145 = KNeighborsRegressor().fit(train_145[predictions], train_145[target])
# use the model to predict values for training set
predictedKNNTrain145 = knn145.predict(train_145[predictions])
samples = train_145.shape[0]
predictors = train_145[predictions].shape[1]
print("\u0332".join("Regression Evaluation Measures for 145 Training Set"))
RegressionMetrics(train_145[target], predictedKNNTrain145, predictors, samples)

In [ ]:
predictedTest = knn145.predict(test_145[predictions])
print("\u0332".join("Regression Evaluation Measures for 145 Test Set"))
RegressionMetrics(test_145[target], predictedTest, predictors, test_145.shape[0])

#### 16

In [ ]:
knn16 = KNeighborsRegressor().fit(train_16[predictions], train_16[target])
# use the model to predict values for training set
predictedKNNTrain16 = knn16.predict(train_16[predictions])
samples = train_16.shape[0]
predictors = train_16[predictions].shape[1]
print("\u0332".join("Regression Evaluation Measures for 16 Training Set"))
RegressionMetrics(train_16[target], predictedKNNTrain16, predictors, samples)

In [ ]:
predictedTest = knn16.predict(test_16[predictions])
print("\u0332".join("Regression Evaluation Measures for 16 Test Set"))
RegressionMetrics(test_16[target], predictedTest, predictors, test_16.shape[0])

### calculate average loss functions for all routes

In [45]:
trips_modelling

,JOURNEY_TIME,temp,weather_main,LINE_DIRECTION,week_sin,week_cos,hour_sin,hour_cos,month_sin,month_cos,rush_hour
0,2924.0,6.39,0,68_1,0.866025,-0.5,-2.449294e-16,1.000000,0.866025,0.500000,0
1,4247.0,-1.61,0,45A_2,0.866025,-0.5,8.169699e-01,-0.576680,0.866025,0.500000,1
2,4020.0,4.39,0,25A_1,0.866025,-0.5,-8.169699e-01,-0.576680,0.866025,0.500000,0
3,4481.0,0.39,0,77A_2,0.866025,-0.5,3.984011e-01,-0.917211,0.866025,0.500000,0
4,6780.0,-1.61,0,39_2,0.866025,-0.5,8.169699e-01,-0.576680,0.866025,0.500000,1
...,...,...,...,...,...,...,...,...,...,...,...
1788429,1363.0,12.39,0,53_2,0.000000,1.0,8.169699e-01,-0.576680,0.500000,-0.866025,1
1788430,1455.0,16.46,0,53_2,0.000000,1.0,-9.790841e-01,0.203456,0.500000,-0.866025,1
1788431,2830.0,11.39,0,45A_1,0.000000,1.0,9.422609e-01,-0.334880,0.500000,-0.866025,1
1788432,3506.0,15.46,0,123_2,0.000000,1.0,-9.422609e-01,-0.334880,0.500000,-0.866025,1


In [46]:
trips_modelling.LINE_DIRECTION.unique()

['68_1', '45A_2', '25A_1', '77A_2', '39_2', ..., '161_1', '16D_1', '33E_1', '41D_1', '41D_2']
Length: 252
Categories (252, object): ['102_1', '102_2', '104_1', '104_2', ..., '84_1', '84_2', '9_1', '9_2']

In [47]:
def modelEvalucation(actualVal, predictions):
    #classification evaluation measures
    print("MAE: ", metrics.mean_absolute_error(actualVal, predictions))
    print("MSE: ", metrics.mean_squared_error(actualVal, predictions))
    print("MAPE:", metrics.mean_absolute_percentage_error(actualVal, predictions))
    print("RMSE: ", metrics.mean_squared_error(actualVal, predictions)**0.5)
    print("R2: ", metrics.r2_score(actualVal, predictions))
    print("\n")

In [48]:
line=trips_modelling.LINE_DIRECTION.unique()
print(line)

['68_1', '45A_2', '25A_1', '77A_2', '39_2', ..., '161_1', '16D_1', '33E_1', '41D_1', '41D_2']
Length: 252
Categories (252, object): ['102_1', '102_2', '104_1', '104_2', ..., '84_1', '84_2', '9_1', '9_2']


In [49]:
MAPEs = []
R2 = []
X_train_size_avg = 0
X_test_size_avg = 0
for i in line:
    dt=trips_modelling[trips_modelling.LINE_DIRECTION ==i]
    dt=dt.drop('LINE_DIRECTION', axis=1)
    y=dt["JOURNEY_TIME"] 
    x=dt.drop('JOURNEY_TIME', axis=1)
    features=x.columns.values.tolist()
    
    X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=3)
    rfc = RandomForestRegressor(n_estimators=50, max_features='auto', random_state=90)
    rfc.fit(X_train, y_train)
     
    #add to avg size of train/test arrays
    X_train_size_avg += X_train.shape[0]/len(line)
    X_test_size_avg += X_test.shape[0]/len(line)
    prediction = rfc.predict(X_test)
    #modelEvalucation(y_test, prediction)
    MAPEs += [metrics.mean_absolute_percentage_error(y_test, prediction)]
    R2 += [metrics.r2_score(y_test, prediction)]


/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feature

/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feature

/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feature

/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feature

/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feature

...

/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feature

/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feature

/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feature

/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feature

/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/home/team9/miniconda3/envs/comp47360py39/lib/python3.9/site-packages/sklearn/ensemble/_forest.py:416: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_feature

In [50]:
print('Avgerage train/test size: %.0f/%.0f'%(X_train_size_avg,X_test_size_avg))
print("The minimum MAPE of all predition models is: %f" %min(MAPEs))
print("The maximum MAPE of all predition models is: %f" %max(MAPEs))
print("The minimum R2 of all predition models is: %f" %min(R2))
print("The maximum R2 of all predition models is: %f" %max(R2))

Avgerage train/test size: 4967/2130
The minimum MAPE of all predition models is: 0.047821
The maximum MAPE of all predition models is: 0.455522
The minimum R2 of all predition models is: -5.373594
The maximum R2 of all predition models is: 0.988669


In [51]:
print(sorted(MAPEs))

[0.04782139297645604, 0.05116013023019949, 0.053250173489933904, 0.05372874505237009, 0.055051918846445065, 0.05929650655248447, 0.05937333438549178, 0.06088787652354453, 0.06187328007105111, 0.06283003552809019, 0.06489474468983593, 0.06543926972970938, 0.065539024889522, 0.0672239167446784, 0.06722550830384703, 0.06728889029018177, 0.06732348894919749, 0.06741467632505921, 0.06770312991064308, 0.06929172111330281, 0.07126206583472759, 0.07130586191114363, 0.07149420187016936, 0.07178012779049434, 0.07202100589066171, 0.07250850612294447, 0.07327103196331985, 0.07360369432763572, 0.07392768236656136, 0.07399428423726537, 0.07428621378283203, 0.07470735016249715, 0.07530458314037146, 0.07546777485079076, 0.07590389776441307, 0.076563496192067, 0.07707209280371632, 0.07710396964866048, 0.07786784590784361, 0.07787406407934006, 0.07826871944293703, 0.07837728518574318, 0.078722935384503, 0.07879156437793715, 0.07881443098056709, 0.07891405787815998, 0.07916350454272093, 0.079203395913069

In [52]:
print(sorted(R2))

[-5.3735944862469465, -0.6960947600529162, -0.5706219736008029, -0.5307276866715238, -0.5086216244989088, -0.3468684280202987, -0.346777955221518, -0.3421761683455713, -0.33282290131393166, -0.2541156452465618, -0.24813877106613647, -0.2450494010009987, -0.2399854861984454, -0.2318016300536332, -0.2003499867008356, -0.19437953284138731, -0.1732060549489456, -0.1571795681039847, -0.13440050896888867, -0.09411917745443787, -0.09193489330640547, -0.08229457028339016, -0.0761051649554898, -0.07553326258993387, -0.049834753899593576, -0.04780792873487272, -0.044837137680571626, -0.032673368818138426, -0.014265030450002936, -0.013201822297280952, -0.011503137120982831, -0.009105683557655952, 0.013062760661510464, 0.021709768564401877, 0.04318741255837022, 0.04704556999637888, 0.06612530687984619, 0.09962427920613492, 0.11639269245370021, 0.12175106844384942, 0.12863547179189616, 0.12911314411641628, 0.13023564984270586, 0.13699398161139797, 0.15153299390724917, 0.15225097463389947, 0.1540040

In [53]:
import numpy

In [54]:
print(numpy.mean(MAPEs))

0.09806478618242273


In [55]:
print(numpy.mean(R2))

0.3447570401870454


### comparision
#### cyclical time (month, weekday, hour) with planned time
Avgerage train/test size: 5029/2156  
The minimum MAPE of all predition models is: 0.044678  
The maximum MAPE of all predition models is: 0.254382  
The minimum R2 of all predition models is: -5.170447  
The maximum R2 of all predition models is: 0.985236  

**mean mape: 0.08867376512843934**  
**mean r2: 0.42657890573517815**    
#### cyclical time (month, weekday, hour) without planned time
Avgerage train/test size: 5029/2156  
The minimum MAPE of all predition models is: 0.046060  
The maximum MAPE of all predition models is: 0.457162  
The minimum R2 of all predition models is: -5.518758  
The maximum R2 of all predition models is: 0.985045  
**mean mape: 0.10144341274223777  
mean r2: 0.3165747676404266**  

#### cyclical time with holiday(almost change nothing,abandon)
Avgerage train/test size: 5029/2156  
The minimum MAPE of all predition models is: 0.047169    
The maximum MAPE of all predition models is: 0.458083  
The minimum R2 of all predition models is: -5.916471  
The maximum R2 of all predition models is: 0.985350  
**mean mape: 0.1014151016387782  
mean r2: 0.31502608722684255**  
#### cyclical time with rush hour 
Avgerage train/test size: 5029/2156  
The minimum MAPE of all predition models is: 0.047886  
The maximum MAPE of all predition models is: 0.455662  
The minimum R2 of all predition models is: -5.916471  
The maximum R2 of all predition models is: 0.984990  
**mean mape: 0.10048396588287027  
mean r2:  0.3232957017842993**
#### what if add holiday to all rush hour?? useless abandon

#### remove outlier from trips set from 1200 to 1800
when 1200:  
Avgerage train/test size: 4730/2028  
The minimum MAPE of all predition models is: 0.036629  
The maximum MAPE of all predition models is: 0.433387    
The minimum R2 of all predition models is: -17.522189  
The maximum R2 of all predition models is: 0.988239  
**mean mape: 0.09363501886411045   
mean r2: 0.29445252397059696**  
when 1800:  
Avgerage train/test size: 4967/2130  
The minimum MAPE of all predition models is: 0.047821  
The maximum MAPE of all predition models is: 0.455522  
The minimum R2 of all predition models is: -5.373594  
The maximum R2 of all predition models is: 0.988669  
**mean mape: 0.09806478618242273  
mean r2: 0.3447570401870454**  

## save pickle files

In [ ]:
# for i in line:
#     dt=trips_modelling[trips_modelling.LINE_DIRECTION ==i]
#     dt=dt.drop('LINE_DIRECTION', axis=1)
#     y=dt["JOURNEY_TIME"] 
#     x=dt.drop('JOURNEY_TIME', axis=1)
#     features=x.columns.values.tolist()
#     X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=3)
#     rfc = RandomForestRegressor(n_estimators=50, max_features='auto', random_state=90)
#     rfc.fit(X_train, y_train)
#     #save model
#     t=i+'.pickle'
#     f = open(t,'wb')
#     pickle.dump(rfc,f)
#     f.close()

In [ ]:
cd ~

In [ ]:
cd pickle

In [ ]:
cd byline

In [ ]:
# pickle testing
f = open('111_1.pickle', 'rb')
model = pickle.load(f)
print(model)

In [ ]:
data = {
        'temp':  6,
        'weather_main': 0,
        'planned_journey_time':44,
        'week_sin': 0.68,     
        'week_cos': 1.0,         
        'hour_sin':  8.16,    
        'hour_cos':  -0.57,
        'month_sin': 0.86,    
        'month_cos': 0.5      
       }
df = pd.DataFrame([data])

In [ ]:
print(df)

In [ ]:
y = model.predict(df)

In [ ]:
print(y)